In [10]:
import re
import pandas as pd
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_ollama import ChatOllama
import logging

logging.basicConfig(level=logging.ERROR)

class FinancialChatbot:
    def __init__(self, trades_path, holdings_path):
        self.trades_path = trades_path
        self.holdings_path = holdings_path
        self.engine = self._init_database()
        self.agent = self._init_agent()

    def _init_database(self):
        print("⚙️ Initializing Database & Cleaning Data...")

        # ---------- TRADES ----------
        df_trades = pd.read_csv(
            self.trades_path,
            na_values=["NULL", "null", ""],
            keep_default_na=True
        )

        if {"TradeDate", "SettleDate"}.issubset(df_trades.columns):
            date_cols = ["TradeDate", "SettleDate"]
        else:
            date_cols = [c for c in df_trades.columns if re.search("date", c, re.I)]

        for col in date_cols:
            df_trades[col] = pd.to_datetime(df_trades[col], errors="coerce")

        # ---------- HOLDINGS ----------
        df_holdings = pd.read_csv(
            self.holdings_path,
            na_values=["NULL", "null", ""],
            keep_default_na=True
        )

        known_dates = [c for c in ["AsOfDate", "InceptionDate", "HoldingDate", "TradeDate"] if c in df_holdings.columns]
        holdings_date_cols = known_dates if known_dates else [
            c for c in df_holdings.columns if re.search("date", c, re.I)
        ]

        for col in holdings_date_cols:
            df_holdings[col] = pd.to_datetime(
                df_holdings[col], dayfirst=True, errors="coerce"
            )

        # ---------- DATABASE ----------
        engine = create_engine("sqlite:///financial.db")
        df_trades.to_sql("trades", engine, index=False, if_exists="replace")
        df_holdings.to_sql("holdings", engine, index=False, if_exists="replace")

        print("✅ Database ready (in-memory SQLite)")
        return engine

    def _init_agent(self):
        db = SQLDatabase(self.engine)

        llm = ChatOllama(
            model="gpt-oss:120b-cloud",
            temperature=0,
            num_ctx=8192
        )

        system_instruction = """
You are an expert Financial Data Analyst with access to two tables: 'trades' and 'holdings'.

RULES:
1. TradeDate is corrupted (NULL/NaT).
   - If user asks date/year-based trade questions → reply EXACTLY:
     "Sorry can not find the answer"

2. Performance / P&L:
   - Use holdings.PL_YTD
   - Rank funds by SUM(PL_YTD) DESC

3. Strict fallback:
   - If data is missing or unavailable → reply EXACTLY:
     "Sorry can not find the answer"

4. Use SQL aggregation only (COUNT, SUM, AVG).
"""

        return create_sql_agent(
            llm=llm,
            db=db,
            agent_type="openai-tools",
            verbose=True,
            prefix=system_instruction
        )

    def ask(self, question: str) -> str:
        try:
            result = self.agent.invoke(question)
            if isinstance(result, dict):
                return result.get("output") or result.get("result") or str(result)
            return str(result)
        except Exception:
            return "Sorry can not find the answer"

In [11]:
bot = FinancialChatbot(
    trades_path="trades.csv",
    holdings_path="holdings.csv"
)

print("🤖 Financial Chatbot Ready")


⚙️ Initializing Database & Cleaning Data...
✅ Database ready (in-memory SQLite)
🤖 Financial Chatbot Ready


In [12]:
questions = ["Which top 3 funds have the highest total PL_YTD?"]
for q in questions:
    print("Q:", q)
    print("A:", bot.ask(q))
    print("-" * 60)


Q: Which top 3 funds have the highest total PL_YTD?


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


holdings, trades
Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


holdings, trades
Invoking: `sql_db_schema` with `{'table_names': 'holdings'}`



CREATE TABLE holdings (
	"AsOfDate" DATETIME, 
	"OpenDate" TEXT, 
	"CloseDate" TEXT, 
	"ShortName" TEXT, 
	"PortfolioName" TEXT, 
	"StrategyRefShortName" TEXT, 
	"Strategy1RefShortName" TEXT, 
	"Strategy2RefShortName" TEXT, 
	"CustodianName" TEXT, 
	"DirectionName" TEXT, 
	"SecurityId" BIGINT, 
	"SecurityTypeName" TEXT, 
	"SecName" TEXT, 
	"StartQty" FLOAT, 
	"Qty" FLOAT, 
	"StartPrice" FLOAT, 
	"Price" FLOAT, 
	"StartFXRate" FLOAT, 
	"FXRate" FLOAT, 
	"MV_Local" FLOAT, 
	"MV_Base" FLOAT, 
	"PL_DTD" FLOAT, 
	"PL_QTD" FLOAT, 
	"PL_MTD" FLOAT, 
	"PL_YTD" FLOAT
)

/*
3 rows from holdings table:
AsOfDate	OpenDate	CloseDate	ShortName	PortfolioName	StrategyRefShortName	Strategy1RefSh